# Project Trinity

#### University of Pavia

- Michele Ventimiglia
- Chiara Curgu
- Bianca Mazzaglia

## Setup

Install required libraries

In [1]:
#!python -m pip install --quiet numpy scipy matplotlib ipykernel ipywidgets

Libraries import

In [2]:
import sys
import scipy
import numpy as np
import ipykernel as ipk
import ipywidgets as ipw
import matplotlib as mpl
import matplotlib.pyplot as plt
from typing import Tuple
from scipy.integrate import odeint
from ipywidgets import interact, FloatSlider, Button

Versions info

In [3]:
print(f"[i] Python v{sys.version}")
print(f"[i] NumPy v{np.__version__}")
print(f"[i] Matplotlib v{mpl.__version__}")
print(f"[i] SciPy v{scipy.__version__}")
print(f"[i] IPyKernel v{ipk.__version__}")
print(f"[i] IPyWidgets v{ipw.__version__}")

[i] Python v3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]
[i] NumPy v1.26.2
[i] Matplotlib v3.8.2
[i] SciPy v1.11.4
[i] IPyKernel v6.27.1
[i] IPyWidgets v8.1.1


## Model

Morris-Lecar model equations

In [4]:
def morris_lecar(
        V_w: Tuple[float, float],
        t: float,
        I_ext: float,
        C: float,
        g_Ca: float,
        g_K: float,
        g_L: float,
        V_Ca: float,
        V_K: float,
        V_L: float,
        phi: float
    ) -> Tuple[float, float]:
    """
    Morris-Lecar model differential equations.
    
    Args:
        V_w (Tuple[float, float]): Membrane potential (V) and recovery variable (w).
        t (float): Time variable.
        I_ext (float): External current.
        C, g_Ca, g_K, g_L (float): Capacitance and maximum conductances.
        V_Ca, V_K, V_L (float): Reversal potentials.
        phi (float): Temperature factor.
    
    Returns:
        Tuple[float, float]: Derivatives dV/dt and dw/dt.
    """
    
    V, w = V_w
    m_inf = 0.5 * (1 + np.tanh((V + 1) / 15))
    w_inf = 0.5 * (1 + np.tanh(V / 30))
    tau_w = 1 / (phi * np.cosh(V / 60))

    dV_dt = (- g_Ca * m_inf * (V - V_Ca) - g_K * w * (V - V_K) - g_L * (V - V_L) + I_ext) / C
    dw_dt = (w_inf - w) / tau_w

    return [dV_dt, dw_dt]

### Parameters

In [5]:
def default_parameters() -> dict:
    """
    Default parameters and initial conditions for the Morris-Lecar model.
    
    Returns:
        dict: A dictionary of default parameters.
    """
    
    params = {
        "C": 1.0,   # Membrane capacitance
        "g_Ca": 1.0,  # Maximum conductances (mS/cm^2)
        "g_K": 2.0,
        "g_L": 0.5,
        "V_Ca": 1.0,  # Reversal potentials (mV)
        "V_K": -0.7,
        "V_L": -0.5,
        "phi": 0.04,  # Temperature factor
        "V0": -0.5,  # Initial membrane potential
        "w0": 0.1    # Initial recovery variable
    }
    
    return params

Time parameter

In [6]:
t = np.linspace(0, 100, 10000)  # Time array

External current

In [7]:
I_ext = 0.1  # External current

## Processing

Solve ODE

In [8]:
def solve_model(
        I_ext: float,
        g_Ca: float,
        g_K: float
    ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Solves the Morris-Lecar model ODEs with given parameters.
    
    Args:
        I_ext (float): External current.
        g_Ca (float): Calcium conductance.
        g_K (float): Potassium conductance.
    
    Returns:
        Tuple[np.ndarray, np.ndarray]: Arrays of membrane potential (V) and recovery variable (w).
    """
    
    params = default_parameters()
    initial_conditions = [params['V0'], params['w0']]
    
    solution = odeint(
        func = morris_lecar,
        y0 = initial_conditions,
        t = t, 
        args = (
            I_ext,
            params['C'],
            g_Ca,
            g_K,
            params['g_L'], 
            params['V_Ca'],
            params['V_K'],
            params['V_L'],
            params['phi']
        )
    )
    return solution[:, 0], solution[:, 1]

Data Analysis

In [9]:
def analyze_data(
        V: np.ndarray,
        w: np.ndarray,
        t: np.ndarray
    ) -> dict:
    """
    Analyzes the data from the Morris-Lecar model simulation.

    Args:
        V (np.ndarray): Array of membrane potentials.
        w (np.ndarray): Array of recovery variable values.
        t (np.ndarray): Time array.

    Returns:
        dict: A dictionary containing analysis results.
    """
    
    analysis_results = {
        'max_potential': np.max(V),
        'min_potential': np.min(V),
        'max_recovery': np.max(w),
        'min_recovery': np.min(w),
        'threshold_potential': None  # Placeholder, define calculation method
    }

    # Example analysis: Identifying threshold potential (customize as needed)
    threshold_index = np.argmax(V > -0.2)  # Example condition
    analysis_results['threshold_potential'] = V[threshold_index]

    return analysis_results

## Results

In [10]:
global current_plot_state

Function to save the plot

In [11]:
def save_plot(
        filename: str = 'plot'
    ) -> None:
    
    global current_plot_state
    
    if current_plot_state is None:
        print("No plot to save.")
        return
    
    I_ext, g_Ca, g_K, V, w, t = current_plot_state
    
    filename = f"{filename}_I_ext_{I_ext}_g_Ca_{g_Ca}_g_K_{g_K}.png"
    
    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.plot(t, V)
    plt.title('Membrane Potential Over Time')
    plt.xlabel('Time')
    plt.ylabel('Membrane Potential (V)')
    plt.grid()

    plt.subplot(1, 3, 2)
    plt.plot(V, w)
    plt.title('Phase Plane')
    plt.xlabel('Membrane Potential (V)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    plt.subplot(1, 3, 3)
    plt.plot(t, w)
    plt.title('Recovery Variable Over Time')
    plt.xlabel('Time')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    plt.savefig(f"docs/img/{filename}")
    plt.close()
    print(f"Plot saved as {filename}")

Plotting

In [12]:
def interactive_plot(
        I_ext: float,
        g_Ca: float,
        g_K: float
    ) -> None:
    
    global current_plot_state
    
    V, w = solve_model(I_ext, g_Ca, g_K)
    analysis_results = analyze_data(V, w, t)
    
    current_plot_state = (I_ext, g_Ca, g_K, V, w, t)
    
    # Plotting
    plt.figure(figsize=(18, 6))

    # Plot V(t)
    plt.subplot(1, 3, 1)
    plt.plot(t, V)
    plt.title('Membrane Potential Over Time')
    plt.xlabel('Time')
    plt.ylabel('Membrane Potential (V)')
    plt.grid()

    # Plot phase plane trajectory
    plt.subplot(1, 3, 2)
    plt.plot(V, w)
    plt.title('Phase Plane')
    plt.xlabel('Membrane Potential (V)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    # Plot w(t)
    plt.subplot(1, 3, 3)
    plt.plot(t, w)
    plt.title('Recovery Variable Over Time')
    plt.xlabel('Time')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()
    
    plt.show()

    # Print analysis results
    print(f"Max Membrane Potential: {analysis_results['max_potential']:.2f}")
    print(f"Min Membrane Potential: {analysis_results['min_potential']:.2f}")
    print(f"Max Recovery Variable: {analysis_results['max_recovery']:.2f}")
    print(f"Min Recovery Variable: {analysis_results['min_recovery']:.2f}")
    print(f"Threshold Potential (Estimate): {analysis_results['threshold_potential']:.2f}")


Interact

In [14]:
interact(
    interactive_plot,
    I_ext = FloatSlider(
        value = 0.1,
        min = -1,
        max = 1,
        step = 0.1,
        description = 'Ext. Current:',
        continuous_update = False
    ),
    
    g_Ca = FloatSlider(
        value = 1.0,
        min = 0,
        max = 5,
        step = 0.1,
        description = 'g_Ca:',
        continuous_update = False
    ),
    
    g_K = FloatSlider(
        value = 2.0,
        min = 0,
        max = 5,
        step = 0.1,
        description = 'g_K:',
        continuous_update = False
    )
)

save_button = Button(description="Save Plot")
save_button.on_click(lambda b: save_plot())
display(save_button)

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='Ext. Current:', max=1.0, mi…

Button(description='Save Plot', style=ButtonStyle())

Plot saved as plot_I_ext_0.1_g_Ca_1.0_g_K_2.0.png
